## Importation des modules

In [1]:
import cv2
import re
import numpy as np
import markdown
from html.parser import HTMLParser
import skvideo
skvideo.setFFmpegPath(r'C:\ProgramData\chocolatey\lib\ffmpeg\tools\ffmpeg\bin')
import skvideo.io
import skvideo.datasets
import skvideo.utils
import subprocess    
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup as Parse
from fractions import Fraction
from matplotlib import colors
import random
import os

## Chargement de la vidéo

In [2]:
IN_PATH = "./videos/snowman_cut.mp4"
OUT_PATH = "./videos/snowman_track.mp4"
OUT_AUDIO_PATH = "./videos/snowman_track_audio.mp4"

# Read video metadata
metadata = skvideo.io.ffprobe(IN_PATH)
#print(metadata)

fps = int(Fraction(metadata['video']['@avg_frame_rate']))
width = int(metadata['video']['@width'])
height = int(metadata['video']['@height'])
length = int(float(metadata['video']['@duration_ts']))
seconds = int(float(metadata['video']['@duration']))

videoiter = skvideo.io.vreader(IN_PATH)

print(f"fps : {fps}    Width: {width}    Height: {height}    Duration: {seconds}s")

fps : 25    Width: 640    Height: 360    Duration: 1536s


## Lecture du fichier description

In [3]:
with open("snowman.md","r",encoding="utf-8") as file:
    text = file.read()

html = markdown.markdown(text)
#display(HTML(html))

In [4]:
def struct(data):
    regex = re.search('(\d+:\d+:\d+)\s*(.*)(?: /(\w+))|(\d+:\d+:\d+)\s*(.*)', data)
    if not regex:
        return
    timecode = regex.group(1) if regex.group(1) else regex.group(4)
    text = regex.group(5) if regex.group(5) else regex.group(2)
    color = regex.group(3)
    if not color:
        color = colors.to_rgb(random.choice(COLORS))
    else:
        color = colors.to_rgb(color)
    color = [int(col*255) for col in color]
    M, s, m = timecode.split(':')
    framecount = int(int(M)*60*fps + int(s)*fps + int(m)*fps/10)
    return dict(start=framecount, text = text, color=[color[2],color[1],color[0]])

def add_duration(data):
    new = []
    for i, cue in enumerate(data, 1):
        if i < len(data):
            cue['duration'] = data[i]['start'] - cue['start']
        else:
            cue['duration'] = length - cue['start']

        new.append(cue)
    return new

In [5]:
WHITE = (255,255,255)
PURPLE = (255,255,0)
BLACK = (0,0,0)
GREEN = (0,255,0)
# https://matplotlib.org/stable/_images/sphx_glr_named_colors_003.png
COLORS = list(colors.cnames.keys())
font = cv2.FONT_HERSHEY_DUPLEX
stream_duration = fps*2

In [6]:
parser = Parse(html)

streamers = [struct(stream.text) for stream in parser.find_all('li') if struct(stream.text)]

sections = [struct(section.text) for section in parser.find_all('h1') if struct(section.text)]
sections = add_duration(sections)

subsections = [struct(sub_section.text) for sub_section in parser.find_all('h2') if struct(sub_section.text)]
subsections = add_duration(subsections)

print(streamers)

[{'start': 40, 'text': 'depart piano', 'color': [0, 255, 0]}, {'start': 600, 'text': 'Depart cordes', 'color': [0, 140, 255]}, {'start': 1165, 'text': '', 'color': [0, 255, 0]}, {'start': 1362, 'text': 'Solo flute', 'color': [0, 140, 255]}, {'start': 1555, 'text': 'Dainty', 'color': [0, 140, 255]}, {'start': 1792, 'text': '', 'color': [0, 255, 0]}, {'start': 2217, 'text': '', 'color': [0, 255, 0]}, {'start': 2735, 'text': '', 'color': [0, 255, 0]}, {'start': 2932, 'text': '', 'color': [0, 255, 0]}, {'start': 3295, 'text': 'Solo clarinet', 'color': [0, 140, 255]}, {'start': 3487, 'text': 'crash', 'color': [0, 140, 255]}, {'start': 3597, 'text': '', 'color': [0, 255, 0]}, {'start': 4025, 'text': '', 'color': [0, 255, 0]}, {'start': 4217, 'text': 'Solo flute', 'color': [0, 140, 255]}, {'start': 4812, 'text': 'Solo flute', 'color': [0, 140, 255]}, {'start': 5122, 'text': '', 'color': [0, 255, 0]}, {'start': 5212, 'text': 'Solo Flute', 'color': [0, 140, 255]}, {'start': 5410, 'text': 'Solo 

In [7]:
def imap(value, fromMin, fromMax, toMin, toMax):
    # Figure out how 'wide' each range is
    fromSpan = fromMax - fromMin
    toSpan = toMax - toMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - fromMin) / float(fromSpan)

    # Convert the 0-1 range into a value in the right range.
    return int(toMin + (valueScaled * toSpan))

def frames_to_TC (frames, fps):
    """
    # Hours: Divide frames by 86400 (# of frames in an hour at 24fps). Round down to nearest integer.
    # Minutes: Divide frames by 1440 (# of frames in a minute). This gives you the total number of minutes, which might be 122 for
    #          content that is 2 hours, 2 minutes, but you don't want the hours here. You're only interested in the extra 2 minutes.
    #          Modulo 60 will remove all full hours and return only the remaining minutes.
    # Seconds: frames % 1440 removes all full minutes and returns the number of remaining frames. 
    #          Divide that by 24 to convert to seconds, and round down to nearest integer.
    # Frames:  frames % 1440 removes all full minutes and returns the number of remaining frames. 
    #          Take that number and modulo 24 to removes all full seconds, leaving you with the remaining # of frames.
    # Lastly, take those variables and put them into a string with colons between each one.
    """
    #h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = int(frames % (60*fps) % fps)
    mil = imap(f, 0,25,0,10)
    return f"{m:02}:{s:02}:{mil}"
    #return ( "%02d:%02d:%02d:%02d" % ( h, m, s, f))
    
def rect(im, frompos, size, color, weight):
    (fromx, fromy) = frompos
    (width, height) = size
    sub_img = im[fromy:fromy+height, fromx:fromx+width]
    col_rect = np.full(sub_img.shape, color, np.uint8)

    res = cv2.addWeighted(sub_img, weight, col_rect, 1, 0)

    # Putting the image back to its position
    im[fromy:fromy+height, fromx:fromx+width] = res
    return im

In [8]:
try:
    os.remove(OUT_PATH)
    os.remove(OUT_AUDIO_PATH) 
except FileNotFoundError:
    pass
writer = skvideo.io.FFmpegWriter(OUT_PATH)
videoiter = skvideo.io.vreader(IN_PATH)

sec = sections.copy()
sub = subsections.copy()

current_section = sec.pop(0)
current_subsection = None
if len(sub):
    current_subsection = sub.pop(0)


dot = None
shift = 0
# Read until video is completed
try:
    for f, frame in enumerate(videoiter,0):

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        #print(f, sec[0]['start'], sec[0]['duration'])
        if len(sec) and f > sec[0]['start']:
            current_section = sec.pop(0)

        if len(sub) and f > sub[0]['start']:
            current_subsection = sub.pop(0)

        # STREAMER DRAWING
        # get all streamers less than 2 seconds from current frame
        next_streamers = [streamer for streamer in streamers if streamer['start'] > f > (streamer['start']-stream_duration)]
        for streamer in next_streamers:
            if not dot and f > streamer['start'] - 2:
                dot = streamer
            startx = int((stream_duration+f - streamer['start'])/stream_duration*width)
            frame = rect(frame, (startx,0), (50, height), streamer['color'], 0)

        # DOT DRAWING
        if dot and dot['start'] < f < dot['start'] + fps/20:
            frame = cv2.circle(frame, (int(width/2), int(height/2)), int(height * 0.5), dot['color'], -1)
        elif dot and f > dot['start'] + fps/10:
            dot = None

        # SECTION DRAWING
        frame = rect(frame, (0,0), (width, 50), BLACK, 0.5)
        frame = cv2.putText(frame, current_section['text'], (5, 30),font, 1, (255,255,255), 2)
        frame = cv2.rectangle(frame, (0,45), (int((f-current_section['start'])/current_section['duration']*width),50), GREEN, -1)

        # SUBSECTION DRAWING
        if current_subsection:
            rec_w = 100
            rec_h = 50
            frame = rect(frame, (width-rec_w,55), (width,rec_h), BLACK, 0.5)
            frame = cv2.putText(frame, current_subsection['text'], (width-rec_w+10, 55+rec_h-20),font, 1, (255,255,255), 2)
            if current_subsection['duration']:
                cue_start = current_subsection['start']
                subsection_percent = int((f-cue_start)/current_subsection['duration']*rec_w+width-rec_w)
                frame = cv2.rectangle(frame, (width-rec_w,rec_h+50), (subsection_percent,rec_h+55), PURPLE, -1)

        # TIMECODE DRAWING
        frame = rect(frame, (0,height-25), (80,height), BLACK, 0.8)
        frame = cv2.putText(frame, frames_to_TC(f, fps), (10,height-8),font, 0.5, (255,255,255), 1)

        # Display the resulting frame
        #cv2.imshow('frame', frame)
        #cv2.waitKey(25)
        #if cv2.getWindowProperty('frame', cv2.WND_PROP_VISIBLE) <1: # Press Q on keyboard to  exit
        #    break
        if f%(60*fps)==0:
            print(f"{(f/fps) /seconds*100}%")


        writer.writeFrame(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
except KeyboardInterrupt:
    pass
        
# Closes all the frames
#cv2.destroyAllWindows()
writer.close()
subprocess.run(f"ffmpeg -i {OUT_PATH} -r {fps} -i {IN_PATH} -r {fps} -c copy ./videos/snowman_track_audio.mp4 -r {fps}")

0.0%
3.90625%
7.8125%
11.71875%
15.625%
19.53125%
23.4375%
27.34375%
31.25%
35.15625%
39.0625%
42.96875%
46.875%
50.78125%
54.6875%
58.59375%
62.5%
66.40625%
70.3125%
74.21875%
78.125%
82.03125%
85.9375%
89.84375%
93.75%
97.65625%


CompletedProcess(args='ffmpeg -i ./videos/snowman_track.mp4 -r 25 -i ./videos/snowman_cut.mp4 -r 25 -c copy ./videos/snowman_track_audio.mp4 -r 25', returncode=0)